In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import csv
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np
from collections import deque
import csv
import os
from copy import deepcopy



# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technical but must be specific so the customer understands the project)

In [3]:
#Create a neural network that plays tetris as long as possible

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

In [4]:
data = pd.read_csv("src/data_0.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'src/data_0.csv'

# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

In [5]:
plt.figure(figsize=(10,6))
plt.plot(data["avg_fit"], label="Average Fitness", marker="o")
plt.plot(data["top_fit"], label="Top Fitness", marker="s")
plt.plot(data["elite_fit"], label="Elite Fitness", linestyle="--")
plt.title("Tetris Reinforcement Learning Training Progress")
plt.xlabel("Generation")
plt.ylabel("Fitness Score")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#Highest fitness is around the 23rd generation

NameError: name 'data' is not defined

<Figure size 1000x600 with 0 Axes>

# 4.Prepare the Data


Apply any data transformations and explain what and why


In [6]:
# Using reinforcement learning so don't need parameters just the board
def bool_to_np(board):
    return np.array(board, dtype=np.float32).flatten()

# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


In [7]:
class RL_AI(nn.Module):
    def __init__(self, input_size=240, hidden_sizes=[128, 256, 256, 128], output_size=1, lr=0.001):
        super(RL_AI, self).__init__()
        layers = []
        last_size = input_size
        for h in hidden_sizes:
            layers.append(nn.Linear(last_size, h))
            layers.append(nn.ReLU(inplace=True))
            last_size = h
        layers.append(nn.Linear(last_size, output_size))
        self.model = nn.Sequential(*layers)
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss_fn = nn.MSELoss()
        self._create_weights()

    def _create_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32)
        if x.ndim == 1:
            x = x.unsqueeze(0)
        return self.model(x)

    def valuate(self, board_np):
        board_tensor = torch.tensor(board_np, dtype=torch.float32).unsqueeze(0)
        return self.forward(board_tensor).item()

    def get_best_move(self, board, piece):
        best_x = -1
        max_value = -float('inf')
        best_piece = None

        for i in range(4):
            piece = piece.get_next_rotation()
            for x in range(board.width):
                try:
                    y = board.drop_height(piece, x)
                except:
                    continue

                board_copy = deepcopy(board.board)
                for pos in piece.body:
                    board_copy[y + pos[1]][x + pos[0]] = True

                np_board = bool_to_np(board_copy)
                value = self.valuate(np_board)

                if value > max_value:
                    max_value = value
                    best_x = x
                    best_piece = piece

        return best_x, best_piece

    def train_step(self, state, target_value):
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        target_tensor = torch.tensor([[target_value]], dtype=torch.float32)
        pred = self.forward(state_tensor)
        loss = self.loss_fn(pred, target_tensor)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()